In [1]:
# Import libraries
from openmm.app import *
from openmm import *
from openmm.unit import *
from openmmml import MLPotential
import sys

# Setup
pdb = PDBFile("aaa.pdb")
pdb.topology.setPeriodicBoxVectors(None)
potential = MLPotential('ani2x')
system = potential.createSystem(pdb.topology)

/home/mbowley/miniconda3/envs/ani/lib/python3.9/site-packages/torchani/__init__.py:55: UserWarning: Dependency not satisfied, torchani.ase will not be available
  warnings.warn("Dependency not satisfied, torchani.ase will not be available")
/home/mbowley/miniconda3/envs/ani/lib/python3.9/site-packages/torch/functional.py:1069: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /home/conda/feedstock_root/build_artifacts/pytorch-recipe_1640869844479/work/aten/src/ATen/native/TensorShape.cpp:2156.)
  return _VF.cartesian_prod(tensors)  # type: ignore[attr-defined]


/home/mbowley/miniconda3/envs/ani/lib/python3.9/site-packages/torchani/resources/


# Small System, ~30 atoms

In [16]:
# Import libraries
from openmm.app import *
from openmm import *
from openmm.unit import *
import openmmml
import sys

# Load small alanine tripeptide system
pdb = PDBFile("../pdbs/aaa.pdb")
potential = openmmml.MLPotential('ani2x')
system = potential.createSystem(pdb.topology)

# Setup simulation
integrator = LangevinIntegrator(
    300 * kelvin, 
    1 / picosecond, 
    1.0 * femtosecond,
)
simulation = Simulation(
    pdb.topology,
    system,
    integrator,
)
STEPS = 1000
simulation.reporters.append(
    StateDataReporter(
        sys.stdout, 
        30, 
        totalSteps=STEPS,
        speed=True,
        progress=True,
        remainingTime=True,
    )
)
simulation.context.setPositions(pdb.positions)

# Minimise and run
simulation.minimizeEnergy()
simulation.step(STEPS)
print("done")

/home/matthewb/miniconda/envs/ani/lib/python3.9/site-packages/torchani/resources/
#"Progress (%)","Speed (ns/day)","Time Remaining"
3.0%,0,--
6.0%,29.4,0:02
9.0%,29.7,0:02
12.0%,29.7,0:02
15.0%,29.7,0:02
18.0%,29.6,0:02
21.0%,29.6,0:02
24.0%,29.7,0:02
27.0%,29.7,0:02
30.0%,29.8,0:02
33.0%,29.7,0:01
36.0%,29.7,0:01
39.0%,29.7,0:01
42.0%,29.8,0:01
45.0%,29.8,0:01
48.0%,29.9,0:01
51.0%,30,0:01
54.0%,30,0:01
57.0%,30.1,0:01
60.0%,30.1,0:01
63.0%,30.1,0:01
66.0%,30.2,0:00
69.0%,30.2,0:00
72.0%,30.2,0:00
75.0%,30.2,0:00
78.0%,30.2,0:00
81.0%,30.2,0:00
84.0%,30.2,0:00
87.0%,30.3,0:00
90.0%,30.2,0:00
93.0%,30.2,0:00
96.0%,30.2,0:00
99.0%,30.3,0:00
done


# Create Large Solvated System, Mixed ANI/AMBER, ~2500 atoms

ANI is used for the peptide, and AMBER for the solvent.
ANI cannot be used alone for such a large system, as it runs into memory errors

In [17]:
# Import libraries
from openmm.app import *
from openmm import *
from openmm.unit import *
import openmmml
import sys

# Load solvated, equilibrated system
pdb = PDBFile("../pdbs_equilibrated/aaa_amber.pdb")

amber_system = ForceField(
    'amber14-all.xml',
    'amber14/tip3p.xml'
).createSystem(
    pdb.topology,
    nonbondedMethod=PME,
    nonbondedCutoff=1*nanometer,
    constraints=AllBonds,
    hydrogenMass=4*amu
)

# Select non-water atoms to be simulated by ANI
ani_atoms = [atom.index for atom in pdb.topology.atoms() if atom.residue.name != "HOH"]

# Setup mixed system
potential = openmmml.MLPotential('ani2x')
system = potential.createMixedSystem(
    pdb.topology,
    amber_system,
    ani_atoms
)

# Setup simulation
integrator = LangevinIntegrator(
    300 * kelvin, 
    1 / picosecond, 
    1.0 * femtosecond,
)
simulation = Simulation(
    pdb.topology,
    system,
    integrator,
)
STEPS = 1000
simulation.reporters.append(
    StateDataReporter(
        sys.stdout, 
        30, 
        totalSteps=STEPS,
        speed=True,
        progress=True,
        remainingTime=True,
    )
)
simulation.context.setPositions(pdb.positions)

# Minimise and run
simulation.minimizeEnergy()
simulation.step(STEPS)
print("done")


/home/matthewb/miniconda/envs/ani/lib/python3.9/site-packages/torchani/resources/
#"Progress (%)","Speed (ns/day)","Time Remaining"
3.0%,0,--
6.0%,26.9,0:03
9.0%,27.4,0:02
12.0%,27.7,0:02
15.0%,27.8,0:02
18.0%,27.8,0:02
21.0%,27.8,0:02
24.0%,27.9,0:02
27.0%,27.9,0:02
30.0%,27.9,0:02
33.0%,28,0:02
36.0%,28,0:01
39.0%,28,0:01
42.0%,27.9,0:01
45.0%,27.9,0:01
48.0%,27.9,0:01
51.0%,27.9,0:01
54.0%,28,0:01
57.0%,28,0:01
60.0%,28,0:01
63.0%,28,0:01
66.0%,27.9,0:01
69.0%,27.9,0:00
72.0%,27.9,0:00
75.0%,27.9,0:00
78.0%,27.9,0:00
81.0%,27.9,0:00
84.0%,27.9,0:00
87.0%,27.9,0:00
90.0%,27.9,0:00
93.0%,27.9,0:00
96.0%,27.9,0:00
99.0%,27.9,0:00
done
